In [ ]:
import math
import re
import collections

def format_quotes():

#  inputs: none
# outputs: list of quotes from quotes2.txt of form quote-speaker


    fp = open('quotes2.txt', 'r')
    line = fp.readline()
    quotes_list = []
    while line is not ' ':
        line = line.rstrip()
        new_line = fp.readline()
        new_line = new_line.rstrip()
        appended_line = '"' + line + '"' + " - " + new_line
        quotes_list.append(appended_line)
        line = fp.readline()
    fp.close()

    return quotes_list
def words_from_quote(quote):

#input: quote
#output: list of words

    list_of_words = re.split('\W+', quote.lower().replace('\'',"").replace('"',""))
    return list_of_words




In [ ]:
def words_from_quote(quote):

#input: quote
#output: list of words

    list_of_words = re.split('\W+', quote.lower().replace('\'',"").replace('"',""))
    return list_of_words




In [ ]:
def count_word_occurrence(quote):

#    input: quote
#    output: dictionary with keys word and values number of times word appears in quote

    d = {}
    for word in words_from_quote(quote):
        if word in d:
            d[word] += 1
        else:
            d[word] = 1
    return d
def create_postings_list_dictionary():

#    input: none
#    output: dictionary with quotes as keys and values are a dictionary with words in quote as keys and word count as values

    quotes_list = format_quotes()
    postings_list_dictionary = {}
    for quote in quotes_list:
        postings_list_dictionary[quote] = count_word_occurrence(quote)
    return postings_list_dictionary


In [ ]:
def create_reverse_postings_list_dictionary():

#    input: none
#    output: dictionary with word as keys and values are dictionary of full quotes as keys and count of word in quote as values

    quotes_list = format_quotes()
    reverse_postings_list_dictionary = {}
    for quote in quotes_list:
        word_counts = count_word_occurrence(quote)
        for word, count in word_counts.items():
            if word in reverse_postings_list_dictionary.keys():
                dictionary_for_word = reverse_postings_list_dictionary[word]
                dictionary_for_word[quote] = count
            else:
                reverse_postings_list_dictionary[word] = {quote:count}
    return reverse_postings_list_dictionary


In [ ]:
def term_frequency(word, quote):

#    input: word, quote
#    output: term frequency as float
  
    word_counts = count_word_occurrence(quote)
    term_frequency = float(word_counts[word]) / float(max(word_counts.values()))
    return term_frequency

def inverse_document_frequency(word, quote):
#    input: word, quote
#    output: inverse document frequency as a float  
    full_quotes_list = format_quotes()
    num_quotes = len(full_quotes_list)
    x = create_reverse_postings_list_dictionary()
    num_quotes_containing_word = len(x[word])
    IDF = math.log(float(num_quotes)/num_quotes_containing_word)
    return IDF
     
def TF_IDF(word, quote):
#    input: word, quote
#    output: TF multiplied by IDF to create TF_IDF measure
    x = term_frequency(word, quote)
    y = inverse_document_frequency(word, quote)
    return x * y

def TF_IDF_SUM(words, quote):
    sum_TFIDF = 0
    for word in words:
        sum_TFIDF += TF_IDF(word, quote)
    return sum_TFIDF


In [ ]:
def single_word_search(word):

#    input: word
 #   output: dictionary with full quotes containing that word as key and TF_IDF scores as values

    x = create_reverse_postings_list_dictionary()
    output = x[word]
    for key in output:
        output[key] = TF_IDF(word, key)

    return output


In [ ]:
def multiple_word_search(words):
    x = create_reverse_postings_list_dictionary()
    starting_point = x[words[0]]
    y = collections.OrderedDict(sorted(starting_point.items(), key = lambda t: t[0]))
    midpoint = [0]*len(y)
    for word in words:
        for index, quote in enumerate(y):
            if word in words_from_quote(quote):
                midpoint[index] += 1
    quotes_dict = {}
    for index, item in enumerate(midpoint):
        if item == len(words):
            quotes_dict[y.keys()[index]] = TF_IDF_SUM(words, y.keys()[index])
    return quotes_dict

